# Evaluate Effect of Step Size and Method Choice
We are going to establish the effect of displacement step size and quantum chemistry method on the accuracy of vibrational properties.

In [ ]:
%matplotlib inline
from matplotlib import pyplot as plt
from ase.thermochemistry import IdealGasThermo
from ase.vibrations import VibrationsData, Vibrations
from jitterbug.compare import compare_hessians
from dataclasses import asdict
from pathlib import Path
import seaborn as sns
import pandas as pd
import numpy as np
import shutil
import json
import os

Configure

In [ ]:
molecule_name = 'water'
relax_level = 'b3lyp_cc-pvtz'  # Which water molecule to evaluate. Need both the molecule name and relaxation level
target_method = ('ccsd(t)', 'cc-pvtz')

## Load in Target Hessians
We have it in a JSON file from PubChem

In [ ]:
hessian_paths = list(Path('data/exact/').glob(f'{molecule_name}_*_at_{relax_level}_d=*-ase.json'))
print(f'Found {len(hessian_paths)} hessians for {molecule_name}')

In [ ]:
exact_path = Path(f'data/exact/{molecule_name}_{"_".join(target_method)}_at_{relax_level}_d=0.005-ase.json')
assert exact_path.exists(), f'Missing reference calculation: {exact_path}'
exact_hess = VibrationsData.read(exact_path)

In [ ]:
def load_hessian(path: Path):
    """Load the Hessian and parse the metadata from the filename
    
    Args:
        path: Path to the Hessian
    Returns:
        Dictionary the includes the metadata for the calculation and errors wrt true Hessian
    """
    
    # Get some of the basic information
    method_name, delta = path.name[:-9].rsplit("_d=", 1)
    delta = float(delta)
    _, method, basis = method_name.split("_")[:3]
    
    # Compare to reference
    approx_hess = VibrationsData.read(path)
    compare = compare_hessians(exact_hess.get_atoms(), exact_hess.get_hessian_2d(), approx_hess.get_hessian_2d(), scale_factor=1.)
    
    return {
        'method': method,
        'basis': basis,
        'title': (method + '//' + basis if basis != "None" else method),
        'delta': delta,
        'path': path,
        **asdict(compare)
    }

In [ ]:
all_hessians = pd.DataFrame(load_hessian(p) for p in hessian_paths)
print(f'Loaded comparisons from {len(all_hessians)} Hessians')

In [ ]:
all_hessians['zpe_error_abs'] = all_hessians['zpe_error'].abs()

Save results

In [ ]:
out_dir = Path('data/error-bounds/')
out_dir.mkdir(exist_ok=True)
all_hessians.to_csv(out_dir / f'{molecule_name}-target={"_".join(target_method)}-methods.csv')

## Summarize Methods

Order the methods by best ZPE

In [ ]:
sorted_methods = all_hessians.groupby('title')['zpe_error_abs'].min().sort_values().reset_index()
sorted_methods.iloc[1:]

In [ ]:
method_order = sorted_methods['title'][1:]

## Plot Performance
Use a heatmap

In [ ]:
def assemble_heatmap(col_name: str) -> tuple[np.ndarray, list[str]]:
    """Assemble the data needed for a heatmap
    
    Args:
        Name of column to be plotted
    Returns:
        Data used for the plot
    """
    
    # Make sure all methods have the same deltas
    deltas = all_hessians.query(f'title != "ccsd(t)//cc-pvtz"').sort_values('delta').groupby('title')['delta'].apply(list)
    assert all(deltas[0] == d for d in deltas), deltas
    
    
    return np.array([
        all_hessians.query(f'title == "{method}"').sort_values('delta')[col_name].tolist()
        for method in method_order
    ]), [str(x) for x in deltas[0]]

In [ ]:
fig, ax = plt.subplots()

zpe_accuracy, xticklabels = assemble_heatmap('zpe_error_abs')
ax = sns.heatmap(zpe_accuracy, cmap="YlGnBu", linewidth=1, annot=True, yticklabels=method_order, xticklabels=xticklabels, ax=ax)
ax.set_title(f'ZPVE error from {"/".join(target_method)} (kcal/mol)')
ax.set_xlabel('Step size')

fig.tight_layout()

In [ ]:
fig, ax = plt.subplots()

zpe_accuracy, xticklabels = assemble_heatmap('vib_mae')
ax = sns.heatmap(zpe_accuracy, cmap="YlGnBu", linewidth=1, annot=True, fmt='.3g', yticklabels=method_order, xticklabels=xticklabels, ax=ax)
ax.set_title(f'Mean Absolute Frequency Error from {"/".join(target_method)} (cm$^{-1}$)')
ax.set_xlabel('Step size')

fig.tight_layout()